In [1]:
import pandas as pd
import eto

In [2]:
df = pd.read_parquet('s3://eto-public/datasets/coco')
df

,image_id,image,file_name,height,width,annotations,retinanet,fasterrcnn,split
0,61779,"{'data': None, 'uri': 'http://images.cocodatas...",000000061779.jpg,425,640,"[{'image_id': 61779, 'area': 26781.77465000000...","[{'bbox': {'xmin': 320.8500061035156, 'ymin': ...","[{'bbox': {'xmin': 320.8500061035156, 'ymin': ...",train
1,382695,"{'data': None, 'uri': 'http://images.cocodatas...",000000382695.jpg,462,640,"[{'image_id': 382695, 'area': 30490.3127499999...","[{'bbox': {'xmin': 382.10430908203125, 'ymin':...","[{'bbox': {'xmin': 382.10430908203125, 'ymin':...",train
2,174866,"{'data': None, 'uri': 'http://images.cocodatas...",000000174866.jpg,640,480,"[{'image_id': 174866, 'area': 557.827000000000...","[{'bbox': {'xmin': 177.25404357910156, 'ymin':...","[{'bbox': {'xmin': 177.25404357910156, 'ymin':...",train
3,159972,"{'data': None, 'uri': 'http://images.cocodatas...",000000159972.jpg,480,640,"[{'image_id': 159972, 'area': 156.978250000000...","[{'bbox': {'xmin': 505.15008544921875, 'ymin':...","[{'bbox': {'xmin': 505.15008544921875, 'ymin':...",train
4,11080,"{'data': None, 'uri': 'http://images.cocodatas...",000000011080.jpg,443,500,"[{'image_id': 11080, 'area': 62551.57474999998...","[{'bbox': {'xmin': 281.0989685058594, 'ymin': ...","[{'bbox': {'xmin': 281.0989685058594, 'ymin': ...",train
...,...,...,...,...,...,...,...,...,...
63995,301981,"{'data': None, 'uri': 'http://images.cocodatas...",000000301981.jpg,480,640,"[{'image_id': 301981, 'area': 44338.6427, 'lab...","[{'bbox': {'xmin': 175.25611877441406, 'ymin':...","[{'bbox': {'xmin': 175.25611877441406, 'ymin':...",val
63996,127955,"{'data': None, 'uri': 'http://images.cocodatas...",000000127955.jpg,425,640,"[{'image_id': 127955, 'area': 70385.8730500000...","[{'bbox': {'xmin': 295.2530212402344, 'ymin': ...","[{'bbox': {'xmin': 295.2530212402344, 'ymin': ...",val
63997,223738,"{'data': None, 'uri': 'http://images.cocodatas...",000000223738.jpg,640,634,"[{'image_id': 223738, 'area': 55136.2386, 'lab...","[{'bbox': {'xmin': 265.9329833984375, 'ymin': ...","[{'bbox': {'xmin': 265.9329833984375, 'ymin': ...",val
63998,244379,"{'data': None, 'uri': 'http://images.cocodatas...",000000244379.jpg,464,640,"[{'image_id': 244379, 'area': 71.0340499999999...","[{'bbox': {'xmin': 86.21955871582031, 'ymin': ...","[{'bbox': {'xmin': 86.21955871582031, 'ymin': ...",val


In [5]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql://rikai:{os.environ["RIKAI_PG_PWD"]}@{os.environ["RIKAI_PG_HOST"]}:5432/postgres?sslmode=disable')

In [ ]:
from io import StringIO
import csv

def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        import time 
        print('Starting')
        start = time.time()
        cur.copy_expert(sql=sql, file=s_buf)
        end = time.time()
        print(f'Took {end - start}')

## One table version

```
CREATE TABLE coco_det (
                          image_id INTEGER PRIMARY KEY,
                          image IMAGE,
                          file_name CHAR(16),
                          height INTEGER,
                          width INTEGER,
                          split VARCHAR(5),
                          annotations JSONB
);
```

In [3]:
df.annotations[0]

array([{'image_id': 61779, 'area': 26781.774650000003, 'label': 'person', 'label_id': 1, 'ann_id': 423148, 'bbox': {'xmin': 123.88, 'ymin': 103.82, 'xmax': 339.24, 'ymax': 424.95}, 'supercategory': 'person'},
       {'image_id': 61779, 'area': 20139.94725000001, 'label': 'person', 'label_id': 1, 'ann_id': 448066, 'bbox': {'xmin': 319.94, 'ymin': 175.73, 'xmax': 575.9, 'ymax': 420.22}, 'supercategory': 'person'},
       {'image_id': 61779, 'area': 285.5689000000001, 'label': 'person', 'label_id': 1, 'ann_id': 1189569, 'bbox': {'xmin': 373.43, 'ymin': 134.66, 'xmax': 386.8, 'ymax': 171.91}, 'supercategory': 'person'},
       {'image_id': 61779, 'area': 106.44479999999996, 'label': 'person', 'label_id': 1, 'ann_id': 1190786, 'bbox': {'xmin': 443.68, 'ymin': 155.37, 'xmax': 454.49, 'ymax': 171.22}, 'supercategory': 'person'},
       {'image_id': 61779, 'area': 172.68535000000006, 'label': 'person', 'label_id': 1, 'ann_id': 1190809, 'bbox': {'xmin': 454.02, 'ymin': 157.42, 'xmax': 476.28, '

In [7]:
import json
coco_det = df[['image_id', 'image', 'file_name', 'height', 'width', 'split', 'annotations']]
coco_det.loc[:, 'image'] = coco_det.image.apply(lambda x: f"({x['uri']})")
coco_det.loc[:, 'annotations'] = coco_det.annotations.apply(lambda x: json.dumps([{'label': el['label'], 'box': el['bbox'], 'supercategory': el['supercategory']} for el in x]))
coco_det

/tmp/ipykernel_4837/675366464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coco_det.loc[:, 'image'] = coco_det.image.apply(lambda x: f"({x['uri']})")
/tmp/ipykernel_4837/675366464.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coco_det.loc[:, 'annotations'] = coco_det.annotations.apply(lambda x: json.dumps([{'label': el['label'], 'box': el['bbox'], 'supercategory': el['supercategory']} for el in x]))


,image_id,image,file_name,height,width,split,annotations
0,61779,(http://images.cocodataset.org/train2017/00000...,000000061779.jpg,425,640,train,"[{""label"": ""person"", ""box"": {""xmin"": 123.88, ""..."
1,382695,(http://images.cocodataset.org/train2017/00000...,000000382695.jpg,462,640,train,"[{""label"": ""person"", ""box"": {""xmin"": 387.25, ""..."
2,174866,(http://images.cocodataset.org/train2017/00000...,000000174866.jpg,640,480,train,"[{""label"": ""book"", ""box"": {""xmin"": 232.16, ""ym..."
3,159972,(http://images.cocodataset.org/train2017/00000...,000000159972.jpg,480,640,train,"[{""label"": ""cup"", ""box"": {""xmin"": 38.72, ""ymin..."
4,11080,(http://images.cocodataset.org/train2017/00000...,000000011080.jpg,443,500,train,"[{""label"": ""dog"", ""box"": {""xmin"": 0.0, ""ymin"":..."
...,...,...,...,...,...,...,...
63995,301981,(http://images.cocodataset.org/val2017/0000003...,000000301981.jpg,480,640,val,"[{""label"": ""giraffe"", ""box"": {""xmin"": 184.2, ""..."
63996,127955,(http://images.cocodataset.org/val2017/0000001...,000000127955.jpg,425,640,val,"[{""label"": ""bear"", ""box"": {""xmin"": 310.01, ""ym..."
63997,223738,(http://images.cocodataset.org/val2017/0000002...,000000223738.jpg,640,634,val,"[{""label"": ""person"", ""box"": {""xmin"": 273.26, ""..."
63998,244379,(http://images.cocodataset.org/val2017/0000002...,000000244379.jpg,464,640,val,"[{""label"": ""car"", ""box"": {""xmin"": 286.15, ""ymi..."


In [8]:
coco_det.to_sql(
    name="coco_det",
    con=engine,
    index=False,
    if_exists="append",
    method=psql_insert_copy
)

Starting
Took 6.814159393310547


## Two table version

```
CREATE TABLE coco (
                      image_id INTEGER PRIMARY KEY,
                      image IMAGE,
                      file_name CHAR(16),
                      height INTEGER,
                      width INTEGER,
                      split VARCHAR(5)
);

DROP TABLE coco_gt;
CREATE TABLE coco_gt (
    id SERIAL PRIMARY KEY,
    image_id INTEGER,
    label varchar(100),
    box BOX,
    supercategory varchar(100),
    CONSTRAINT fk_image_id
        FOREIGN KEY(image_id)
            REFERENCES coco(image_id)
);
```

In [10]:
images = df[['image_id', 'image', 'file_name', 'height', 'width', 'split']]
images.loc[:, 'image'] = images.image.apply(lambda x: f"({x['uri']})")
images.to_sql(
    name="coco",
    con=engine,
    index=False,
    if_exists="append",
    method=psql_insert_copy
)

/tmp/ipykernel_4837/660500096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  images.loc[:, 'image'] = images.image.apply(lambda x: f"({x['uri']})")


Starting
Took 1.4307730197906494


In [20]:
ann = pd.json_normalize(df.annotations.explode(), max_level=0)
ann = ann[['ann_id', 'image_id', 'label', 'bbox', 'supercategory']].rename({'ann_id': 'id', 'bbox': 'box'}, axis=1)
ann.loc[:, 'box'] = ann.box.apply(lambda x: f'(({x["xmin"]}, {x["ymin"]}), ({x["xmax"]}, {x["ymax"]}))')
ann

,id,image_id,label,box,supercategory
0,423148,61779,person,"((123.88, 103.82), (339.24, 424.95))",person
1,448066,61779,person,"((319.94, 175.73), (575.9, 420.22))",person
2,1189569,61779,person,"((373.43, 134.66), (386.8, 171.91))",person
3,1190786,61779,person,"((443.68, 155.37), (454.49, 171.22))",person
4,1190809,61779,person,"((454.02, 157.42), (476.28, 174.60999999999999))",person
...,...,...,...,...,...
479885,1347535,244379,car,"((286.15, 294.59), (295.87, 303.19))",vehicle
479886,402737,244379,traffic light,"((86.74, 53.82), (116.0, 110.25))",outdoor
479887,411437,244379,traffic light,"((206.04, 49.73), (233.95999999999998, 106.02))",outdoor
479888,449134,177357,person,"((71.07, 138.9), (389.11, 253.63))",person


In [21]:
ann.to_sql(
    name="coco_dt",
    con=engine,
    index=False,
    if_exists="append",
    method=psql_insert_copy
)

Starting
Took 3.836305856704712
